<img src="images/bwHPC_Logo_cmyk.svg" width="200" /> <img src="images/HochschuleEsslingen_Logo_RGB_DE.png" width="200" /> <img src="images/Konstanz_Logo.svg" width="200" /> <img src="images/KIT_Logo.png" width="200" />

# Dask
Dask builds on proven modules and extends them with options for massive parallelization. For example, several NumPy arrays or Pandas data frames can be combined in corresponding Dask objects and made available for parallel operations. The Dask objects provide large parts of the well-known API (identical to NumPy arrays or Pandas data frames).

Dask can also store data in objects whose size exceeds the available memory. To do this, Dask stores parts of the data in an available file system. Dask can therefore be used to process data volumes that are actually too large for Pandas or NumPy. However, if only a small amount of data needs to be processed/analyzed, the overhead required by Dask can lead to a slowdown compared to pure NumPy/Pandas objects.

![image](images/Dask_Scale.svg)

## Dask Dashboard

An overview of the parallel processes started by Dask and the utilization of the resources reserved via Dask can be viewed via the Dask Dashboard. The client object from the dask.distributed module enables a Dask dashboard to be started. If the initialized client object is output, the output contains a URL under which the started dashboard can be accessed.

If Jupyter version 3.0 is installed or Node.js (version >= 12.0.0) and npm are also installed, the dask-labextensions plugin can be installed in Jupyter as an alternative to manual use. This ensures that the Dask dashboard is integrated into the Jupyter interface. A new "Dask" button is then available on the left-hand side. This can be used to access the Dask dashboard without having to call up a separate URL.

In [ ]:
from dask.distributed import Client
client = Client(processes=False, threads_per_worker=4,
                n_workers=1, memory_limit='2GB')
client

In [ ]:
client.close()

The dask-labextensions plugin does not currently work with dask-mpi on the bwUniCluster. Alternatively, the URL of the dashboard can be used directly as described above. To do this, the port from the URL must be forwarded from the cluster to the outside via ssh. This can be done locally on the computer being used in a console using the following command. The port and IP of the Jupyter compute node can be taken from the dashboard URL.

```bash
ssh -N -L <Port>:<Jupyter-Compute-Node>:<Port> <university abbreviation>_<User-ID>@bwunicluster.scc.kit.edu
```

Example:
```bash
ssh -N -L 8787:10.0.1.112:8787 es_pkoester@bwunicluster.scc.kit.edu
```

After executing the ssh port forwarding, the Dask dashboard can be called up on the local computer at

```bash
http://localhost:<Port>/status
```

## Dask Array

Dask Array coordinates several NumPy arrays and distributes them across the available resources. This allows operations to be carried out across multiple threads, processes or even nodes. Which operations are possible (which parts of the NumPy Array API are also offered by Dask Array) can be found in the documentation: https://docs.dask.org/en/latest/array-api.html.

Further examples of Dask Array: https://mybinder.org/v2/gh/dask/dask-examples/main?urlpath=lab/tree/array.ipynb

## Dask Dataframe

What Dask Array is for NumPy arrays (see above), Dask Dataframe is for Pandas Dataframe. The operations possible on a Dask Dataframe can be found in the documentation: https://docs.dask.org/en/latest/dataframe-api.html.

A general overview of useful and less useful ways of using a Dask Dataframe can be found at https://docs.dask.org/en/latest/dataframe.html.

## Dask and SLURM

In order to use Dask in combination with SLURM (the bwUniCluster job scheduler), either the SLURMCluster class from the dask_jobqueue module or the dask-mpi program is required.

The module dask-mpi is required for the following exercises. dask_jobqueue is therefore only described in basic terms (the environment created in "2_Fundamentals" was only prepared for dask-mpi: the exercises below can therefore only be carried out with dask-mpi).

### dask_jobqueue

__IMPORTANT: dask_jobqueue assumes a one-to-one relationship between job and node. This means that exactly one node is reserved and used per job. If several nodes are required, dask_jobqueue ensures that a corresponding number of jobs are submitted. This leads to two fundamental problems on the bwUniCluster. Firstly, several jobs are scheduled independently of one another. Each job therefore has its own start time. It is only with a lot of luck that all the required resources are available at the same time. On the other hand, the cpu queue allows more than one node to be reserved per job. If this queue is used together with dask_jobqueue, dask_jobqueue sets up a separate job for each required node, which reserves two nodes each, of which dask_jobqueue then only uses one.__

Conclusion:
- in order to have several nodes available at the same time, bwUniCluster provides a job in a queue with several nodes per job
- dask_jobqueue requires that only one node is used per job
- dask-mpi should be used instead of dask_jobqueue

In order for dask_jobqueue to be available, both dask and dask_jobqueue must be installed in the respective environment:

```bash
python3 -m pip install dask_jobqueue dask
```

If the IPython kernel from a correspondingly extended environment is registered in Jupyter, it can be selected when starting a new notebook. The SLURMCluster class can then be imported into the notebook and used to create a SLURM job configuration.

Which queues are available for such a configuration on the bwUniCluster and what properties they have can be found in the documentation at

https://wiki.bwhpc.de/e/BwUniCluster3.0/Running_Jobs#Queues_on_bwUniCluster_3.0

In [ ]:
from dask_jobqueue import SLURMCluster
cluster = SLURMCluster(
    queue='cpu',            # queue cpu allows up to 20 nodes per job 
    cores=192,              # a node that has queue cpu has 96 cores => two nodes are required for  cores
    memory="380GB",       # maximum available memory per node in queue cpu
    local_directory='/tmp', # data should be written locally in the node and not to the central file system via the network
    walltime='00:30:00',    # nodes should be reserved for half an hour
    interface='ib0'         # we want to use fast Infiniband for network communication in the cluster
)

The actual job is then started based on the configuration using the scale method:

In [ ]:
cluster.scale(jobs=1) # when starting the configuration, several jobs can be started simultaneously (this makes it possible to reserve several nodes)

In [ ]:
from dask.distributed import Client
client = Client(cluster)
client # contains information on the started cluster

In [ ]:
client.close()

### dask-mpi

The dask-mpi application enables you to start a dask cluster via MPI. This makes it possible to reserve several nodes with just one job. This also makes it possible to use queues that require more than one node per job. It also ensures that all required nodes are available at the same time (since they were requested via the same job).

#### Environment

The environment created in the "2_Basics" notebook is required for the following examples. This environment must be selected for this notebook via the corresponding kernel.

So that the environment created in the notebook "2_Basics" can be quickly copied to the nodes used, it should be packed in an archive (the following command must be executed in the terminal File->New->Terminal):

```bash
tar -zcvf ~/miniconda3/envs/python_workshop_env.gz -C ~/miniconda3/envs/ python_workshop_env/
```

#### sbatch and job script

To start a dask cluster via dask-mpi, a job script is required. This is started later via sbatch. Sbatch ensures that the required number of nodes is reserved (this also specifies how many tasks should be started and how they should be distributed across the nodes). Once the requested nodes are available, sbatch executes the script passed on the first of the reserved nodes. In this script, the required data can then be copied to the individual nodes (via separate jobs that are started via srun; local data can be read in more quickly than accessing the central file system). Finally, the script executes a call to mpirun. Dask-mpi is started n times via mpirun. n should be equal to the number of workers required plus one (for the scheduler). The number n must match the number of tasks configured via sbatch.

#### Number of processes per node and number of threads per process:

The number of threads per process per node multiplied by the number of processes should equal the sum of the available cores. This is the only way to ensure that all threads have a core available and are not slowed down by other threads. However, if individual tasks involve waiting times (e.g. waiting for file I/O or waiting for results from other tasks), it may be better to plan for more parallel tasks (processes or threads) than cores are available.

Information about the queues: https://wiki.bwhpc.de/e/BwUniCluster3.0/Running_Jobs#Queues_on_bwUniCluster_3.0

Information about the hardware per node: https://wiki.bwhpc.de/e/BwUniCluster3.0/Hardware_and_Architecture

If numerical libraries are used predominantly, a high number of threads per process makes sense, as these libraries (Numpy, Pandas, ...) are implemented in such a way that they are not slowed down by Python's Global Interpreter Lock (GIL). They are usually written in C and only offer an interface for access from Python. At the same time, they benefit from the shared use of the same data within a process (no interprocess communication between threads in the same process necessary).

If algorithms written mainly in Python are executed, many processes with few or even just one thread per process are a good idea, as the GIL prevents parallel execution with multiple threads in one process.

Mixtures of numerical libraries and algorithms implemented in Python require a more balanced ratio between processes and threads.

In [ ]:
import os

## For reservation queue:
##  - multi, count_nodes must be at least 2
##  - single, count_nodes must be exactly 1
count_nodes = 2
queue = "cpu"

count_worker_per_node = 2     # the scheduler is started on the first node => the first node provides one less worker
count_threads_per_worker = 20 # count_worker_per_node * count_threads_per_worker should correspond to the number of cores available per node (see above)
time = "30:00"                # 30 minutes of runtime
mem = "90000mb"
tmp_envs_dir = f"$TMP/envs"   # $TMP refers to a local directory for each node and job (/scratch/slurm_tmpdir/job_<job-id>)
tmp_dask_dir = f"$TMP/dask"   # Dask should swap data under $TMP if there is not enough memory

scheduler_file = os.path.expanduser("~/dask-scheduler.json")

In [ ]:
job_file = os.path.expanduser("~/job_dask_mpi.sh")

f = open(job_file, "w") # create a new script that can then be sent via sbatch
f.write(f"""#!/bin/bash -l
# -l is used to adopt the settings of the bashrc (required for conda)

# Variable to go through each node reserved via sbatch individually in a for loop
NODES=$(scontrol show hostname | cat)
DASK_HOSTS=""
for NODE in $NODES; do
    # In single queues the node list required for mpirun is missing,
    # so we use a self-created host list.
    # The double curly brackets are the escape sequence for single curly brackets
    # within a Python format string. DASK_HOSTS+="${{NODE}},"

    # start a job per node using srun and save data locally in the node under $TMP
    # the "&" at the end ensures that the srun is executed in parallel in the background
    # the "\" at the end of the line masks the target end (the next line still belongs to the current line)
    # rm -rf deletes a directory
    # mkdir creates a directory
    # cp copies a file
    # tar unpacks the file
    # the "&&" between the individual commands ensures that the following commands are only executed
    # if the previous command was successful (it did not return an error)
    srun -N 1 -n 1 -w $NODE /bin/bash -c "\
      rm -rf {tmp_envs_dir} && \
      rm -rf {tmp_dask_dir} && \
      mkdir -p {tmp_envs_dir} && \
      mkdir -p {tmp_dask_dir} && \
      cp ~/miniconda3/envs/python_workshop_env.gz {tmp_envs_dir} && \
      tar -zxf {tmp_envs_dir}/python_workshop_env.gz --directory {tmp_envs_dir} \
    " &
done

# wait for all commands started in parallel

wait

# Activate the local environment (does not have to be done on every node, as the
# corresponding environment variables are adopted via mpirun)

conda activate {tmp_envs_dir}/python_workshop_env

# starts the workers and a scheduler on the nodes
# --map-by core:PE=n: each process should be assigned n cores
# -np x: x = number of workers + 1 for scheduler (x must be equal to --ntasks when executing sbatch)
# -host=$DASK_HOSTS: hosts on which processes should be started
# --scheduler-file: as soon as all workers are started, the scheduler writes the connection information of the dask cluster to this file)
# --interface='ib0': we use Infiniband for communication between the nodes
# --local-directory={tmp_dask_dir}: if dask has to outsource data due to a lack of memory, the local file system of the respective node should be used for this
# --worker-class=distributed.Worker: no monitoring nanny process per worker process
# --nthreads: threads per worker
# --name: prefix for naming the workers in this job
# --dashboard-address: the port under which the dashboard can be reached (8787 is actually the Default value, but in the current development state this is not set automatically)
mpirun --map-by core:PE={count_threads_per_worker} \
    -np {count_nodes * count_worker_per_node} \
    -host=$DASK_HOSTS \
    dask-mpi \
    --scheduler-file {scheduler_file} \
    --interface='ib0' \
    --local-directory={tmp_dask_dir} \
    --worker-class=distributed.Worker \
    --nthreads={count_threads_per_worker} \
    --name base \
    --dashboard-address=8787""")
f.close()

In [ ]:
# how many nodes are currently freely available per queue
# can also be issued in a terminal (File->New->Terminal) outside the Jupyter Notebook as the command "sinfo_t_idle"
os.system("sinfo_t_idle")

In [ ]:
# rm: delete the ~/dask-scheduler.json file: as soon as the file is back, the requested Dask cluster is started
# sbatch: reserves the necessary resources (nodes) in the specified queue and starts the script ~/job_dask_mpi.sh on the first of the nodes
os.system(f"rm -f {scheduler_file} && \
            sbatch \
            -p {queue} \
            --nodes={count_nodes} \
            --ntasks={count_nodes * count_worker_per_node} \
            --ntasks-per-node={count_worker_per_node} \
            --time={time} \
            --mem={mem} \
            {job_file}")

In [ ]:
# Has the queueing system provided a tentative start time of the job?
os.system("squeue --start")

In [ ]:
# Which resources are requested for our user?
# Shows one line per job with which resources were requested.
#   ST: Status
#   PD: Pending, resources have been requested but are not yet available
#   R: Running, resources are available
#   CG: Completing, job is finished/aborted, but individual processes are still running (which are still being finished or aborted)
#   other status codes: https://curc.readthedocs.io/en/latest/running-jobs/squeue-status-codes.html
#   TIME: how long have the resources been used by us
#   NODES: number of reserved nodes
#   NODELIST: which nodes are reserved
# The name of the nodes is given with a prefix (is the same for each node)
# followed by the numbering of the individual nodes in square brackets.
# A hyphen between two numbers in the square brackets means that all
# numbers between the two given are reserved for us.
# A comma between two numbers means that the two numbers are reserved for us.
# Examples:
#   uc3n[001-003] indicates that uc3n001, uc3n002 and uc3n003 have been reserved
#   uc3n[001,003] indicates that uc3n001 and uc3n003 have been reserved
os.system("squeue")

In [ ]:
import os, time as t

# Wait until the scheduler_file (~/dask-scheduler.json) is available
# (Will be written by the dask-scheduler as soon as worker has been started and is ready=
while not os.path.isfile(scheduler_file):
    t.sleep(1)

In [ ]:
# Current status and error messages of the SLURM job may be retrieved from the slurm-<JOBID>.out file:
jobid = 22439085      # PLEASE ADAPT THE JOBID as described above
cwd = os.getcwd()
f = open(cwd + "/slurm-" + str(jobid) + ".out","r")
lines = f.readlines()
display(lines)

In [ ]:
import dask
from dask.distributed import Client

# We can use a client object to connect the current Jupyter notebook to the created Dask cluster.
# To do this, we use the ~/dask-scheduler.json file. This is where the scheduler's IP and port are stored.
# (instead of from a Jupyter notebook, this can also be done from a Python script, for example)
client = Client(scheduler_file=scheduler_file)

# if dask has to swap out data due to a lack of memory,
# the local file system of the respective node should be used
dask.config.set({'temporary_directory': tmp_dask_dir})

In [ ]:
# Let's show the client (which contains information on the scheduler and the workers)
client

### Enlarge Dask cluster later

Additional workers can be created with a separate job and added to the scheduler from a previous job. The scheduler-json file is also used for this.

In [ ]:
job_file = os.path.expanduser("~/job_dask_mpi_2.sh")

# A script to enlarge an existing Dask -Cluster
f = open(job_file, "w")
f.write(f"""#!/bin/bash -l
NODES=$(scontrol show hostname | cat)
DASK_HOSTS=""
for NODE in $NODES
do
    DASK_HOSTS+="${{NODE}},"
    
    srun -N 1 -n 1 -w $NODE /bin/bash -c "\
        rm -rf {tmp_envs_dir} && \
        rm -rf {tmp_dask_dir} && \
        mkdir -p {tmp_envs_dir} && \
        mkdir -p {tmp_dask_dir} && \
        cp ~/miniconda3/envs/python_workshop_env.gz {tmp_envs_dir} && \
        tar -zxf {tmp_envs_dir}/python_workshop_env.gz --directory {tmp_envs_dir} \
        " &
done

wait

conda activate {tmp_envs_dir}/python_workshop_env

# Since an existing Dask-Cluster shall be enlarged:
# --no-scheduler: do not create a new scheduler, but rather register the works with the Scheduler specified in the $scheduler_file
mpirun --map-by core:PE={count_threads_per_worker} \
       -np {count_nodes * count_worker_per_node} \
       -host=$DASK_HOSTS \
       dask-mpi \
       --scheduler-file {scheduler_file} \
       --interface='ib0' \
       --local-directory={tmp_dask_dir} \
       --worker-class=distributed.Worker \
       --nthreads={count_threads_per_worker} \
       --no-scheduler \
       --name expansion""")
f.close()

# Reserve another node in the batch system and assign to the cluster
os.system(f"sbatch \
            -p {queue} \
            --nodes={count_nodes} \
            --ntasks={count_nodes * count_worker_per_node} \
            --ntasks-per-node={count_worker_per_node} \
            --time={time} \
            --mem={mem} \
            {job_file}")

In [ ]:
os.system("squeue")

In [ ]:
# Wait until workers are available (for the first worker in the second job)
client.wait_for_workers((count_nodes * count_worker_per_node * 2) -1)

In [ ]:
# Again show information on the workers
client

In [ ]:
# instead of connecting a client to a Dask cluster via the json file, an IP address can also be used
from distributed import Client
# for this, the IP and PORT for registration must be known (e.g. from the slurm-<JOBID>.out file)
client = Client('172.26.20.82:37153')

In [ ]:
client

## Example Dask Array

In [ ]:
import dask.array as da

# Creates an array with 100,000 rows and 100,000 columns.
# Each element contains a random value from the interval [0.0, 1.0).
#
# The array is divided into individual chunks. Each chunk is created by
# a separate task. This allows Dask to distribute each chunk to
# another worker.
# The tasks are only executed when the array is being worked on.
# x therefore does not contain a finished array, but only the tasks
# that are necessary to create the array.
# Instead of specifying the maximum chunk size in MiB, the
# number of elements per chunk can also be defined (see
# https://docs.dask.org/en/latest/array-chunks.html).
x = da.random.random((100000,100000), chunks="100 MiB")
x

In [ ]:
# A calculation operation on a Dask array initially only creates tasks.
# These are only executed when a result is requested.
y = (x + x.T) - x.mean(axis=0)
y

In [ ]:
# The compute() method requests a specific result. By calling this method, the planned tasks are executed.
y.sum().compute()

## Example Dask DataFrame and Scikit-Learn

Based on https://github.com/rikturr/high-performance-jupyter (MIT-License).

In [ ]:
%%time
# %%time measures the time that a cell in the notebook needs for execution
# %%time must be the first line in the cell

import dask.dataframe as dd

# erstellt Tasks um ~8 GB Daten aus einem s3 Bucket zu laden und als Dataframe einzulesen
#taxi = dd.read_csv(
#    's3://nyc-tlc/trip data/yellow_tripdata_2019-*.csv',
#    assume_missing=True, # beim Einlesen werden alle Ints zu Floats. Dies erlaubt fehlende Werte
#    parse_dates=['tpep_pickup_datetime', 'tpep_dropoff_datetime'], # interpretiert diese Spalten als Datum
#    storage_options={'anon': True}, # für S3: keine Authentifizierung für diesen Bucket nötig
#)
## S3 requires an account for AWS; direct access therefore not possible
## Please use the provided parquet-File (see below)

taxi = dd.read_parquet(
    './files/green_tripdata_2023-01.parquet',
    engine='pyarrow'
)
#df = pd.read_parquet('./files/green_tripdata_2023-01.parquet', engine='pyarrow')
taxi

In [ ]:
%%time

# both following lines execute the tasks contained in "taxi",
# to use their results for further operations
# => Reading into the data frame is executed twice!

print(f"Number of rows: {len(taxi)}") #Rows of all data records together
print(f"Size in GB: {taxi.memory_usage(deep=True).sum().compute() / 1e9}")

In [ ]:
from dask.distributed import wait

# persist() executes all planned tasks in the background (asynchronously) and returns the result
# we therefore replace the tasks here with the finished data frame
taxi = taxi.persist()
# wait() waits for the tasks started in the background
_ = wait(taxi)

In [ ]:
%%time

# len() and memory_usage() are significantly faster when working on the persisted
# result of the tasks
print(f"Number of rows: {len(taxi)}") #Rows of all records together
print(f"Size in GB: {taxi.memory_usage(deep=True).sum().compute() / 1e9}")

In [ ]:
numeric_feat = [
    'pickup_weekday', 
    'pickup_hour', 
    'pickup_week_hour', 
    'pickup_minute', 
    'passenger_count',
]
categorical_feat = [
    'PULocationID', 
    'DOLocationID',
]
features = numeric_feat + categorical_feat
y_col = 'high_tip'

In [ ]:
def prep_df(df: dd.DataFrame) -> dd.DataFrame:
    '''
    Generate features from a raw taxi dataframe.
    '''
    df = df[df.fare_amount > 0]  # Only the lines, that contain non-zeros (to avoid division-by-zero)
    df['tip_fraction'] = df.tip_amount / df.fare_amount
    df[y_col] = (df['tip_fraction'] > 0.2) # If tip_amount / fare_amount > 0.2, then high_tip = true
    
    df['pickup_weekday'] = df.lpep_pickup_datetime.dt.weekday
    df['pickup_weekofyear'] = df.lpep_pickup_datetime.dt.isocalendar().week
    df['pickup_hour'] = df.lpep_pickup_datetime.dt.hour
    df['pickup_week_hour'] = (df.pickup_weekday * 24) + df.pickup_hour
    df['pickup_minute'] = df.lpep_pickup_datetime.dt.minute
    
    # Convert all inputs for the training to floats and set unavailable values to -1
    df = df[features + [y_col]].astype(float).fillna(-1)
    
    return df
    
taxi = prep_df(taxi)
taxi

In [ ]:
%%time

# Task to prepare the data frame (prep_df)
taxi = taxi.persist()
_ = wait(taxi)

In [ ]:
taxi.head()

In [ ]:
%%time

import numpy as np

# describe() generates for each columns a statisctical overview (Min., Max., Quantile, ...)
np.round(taxi.describe().compute(), 3).T

In [ ]:
seed = 42

taxi_sample = taxi.sample(frac=0.02, replace=False, random_state=seed)
taxi_sample = taxi_sample.persist()
_ = wait(taxi_sample)

len(taxi_sample)

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from dask_ml.compose import ColumnTransformer
from dask_ml.preprocessing import StandardScaler, DummyEncoder, Categorizer
from dask_ml.model_selection import GridSearchCV

seed = 42

lr = LogisticRegression(
    solver='saga',
    penalty='elasticnet', 
    l1_ratio=0.5,
    max_iter=100, 
    random_state=seed,
)
pipeline = Pipeline(steps=[
    ('categorize', Categorizer(columns=categorical_feat)),
    ('onehot', DummyEncoder(columns=categorical_feat)),
    ('scale', ColumnTransformer(transformers=[('num', StandardScaler(), numeric_feat)])),
    ('clf', lr),
])

params = {
    'clf__l1_ratio': [0.2, 0.3, 0.5, 0.7, 0.9],
}

grid_search = GridSearchCV(
    pipeline, 
    params,
    cv=3, 
    scoring='accuracy',
)

In [ ]:
%%time

# Before fit() is called, joblib can be told that a Dask cluster is available.
# joblib is used by scikit-learn to parallelize. This can be useful if
# objects from dask_ml are not already being used.
#import joblib            # is needed to connect scikit-learn to the Dask cluster
#with joblib.parallel_backend('dask'):
# _ = grid_search.fit(taxi_sample[features], taxi_sample[y_col])
_ = grid_search.fit(taxi_sample[features], taxi_sample[y_col])

grid_search.best_score_

## Functions as tasks

With the "delayed" decorator, functions can be converted into tasks when called. These tasks are then executed by Dask as soon as a result is requested. Scheduling and executing a task generates overhead. A task should therefore always contain as much computing power as possible. It can be counterproductive to keep tasks as small as possible. The following example is only intended to help you understand tasks. It contains tasks that are too small. If the range() in the for loop is increased, problems quickly arise (Dask generates warnings).

Delayed functions are subject to restrictions. For example, they cannot be used in a condition (if or loop condition).

More information on delayed: https://docs.dask.org/en/stable/delayed-api.html

In [ ]:
import dask.delayed as delayed

@delayed
def increment(x):
    return x + 1

@delayed
def power2(x):
    return x ^ 2

@delayed
def add(x, y):
    return x + y

# Generation of 10000 Tasks (each of two Tasks) into a list
output = []
for x in range(1, 10000):
    a = increment(x)
    b = power2(x)
    c = add(a, b)
    output.append(c)

# A new task based on the list
total = delayed(sum)(output)
total

In [ ]:
# Execute Tasks
res = total.compute()
res

## Visualization of Dask-Tasks

In [ ]:
output2 = []
# So that the generated graph stays clear: just 10 Iterations
for x in range(1, 10):
    a = increment(x)
    b = power2(x)
    c = add(a, b)
    output2.append(c)

total2 = delayed(sum)(output2)
total2.visualize()

In [ ]:
res = total2.compute()
res

## Progress Bar

In [ ]:
client.close() # ProgressBar only works if no Dask cluster is actively connected
# if necessary, the cluster job must be terminated so that the ProgressBar is displayed (see Ending jobs below)

from dask.diagnostics import ProgressBar

with ProgressBar():
    res = total.compute()
res

## Free Ressources (end Jobs)

In [ ]:
# If resources are no longer needed, the associated job must be terminated
# to release the resources (nodes) again.
job_id = 22439198               # PLEASE ADAPT JOB_ID
os.system(f"scancel {job_id}")